In [1]:
from pathlib import Path
import pandas as pd

flightlist = pd.concat(
    pd.read_csv(file, parse_dates=["firstseen", "lastseen", "day"])
    for file in Path("C:\\Users\\ncevik\\Desktop\\TİK2\\Flight\\Data\\zip").glob("flightlist_*.csv.gz")
)

C:\Users\ncevik\anaconda3\envs\traffic\lib\site-packages\pandas\core\reshape\concat.py:348: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


In [4]:
flightlist

,callsign,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,HVN19,NaN,888152,NaN,NaN,YMML,LFPG,2018-12-31 00:43:16+00:00,2019-01-01 04:56:29+00:00,2019-01-01 00:00:00+00:00,-37.659485,144.804421,304.8,48.995316,2.610802,-53.34
1,CCA839,NaN,780ad1,NaN,NaN,YMML,LEBL,2018-12-31 00:53:08+00:00,2019-01-01 06:01:42+00:00,2019-01-01 00:00:00+00:00,-37.692123,144.841997,304.8,41.036124,2.063557,1112.52
2,CES219,NaN,780b7e,B-5936,A332,YSSY,EDDF,2018-12-31 01:05:29+00:00,2019-01-01 04:09:29+00:00,2019-01-01 00:00:00+00:00,-33.930908,151.171987,0.0,50.045563,8.588923,-15.24
3,AEA040,NaN,34444e,EC-LVL,A332,LEMD,LEMD,2018-12-31 01:07:21+00:00,2019-01-01 03:32:59+00:00,2019-01-01 00:00:00+00:00,40.534756,-3.575426,609.6,40.475728,-3.538347,411.48
4,CXA825,NaN,780d75,B-2760,B788,YSSY,LFPG,2018-12-31 01:18:29+00:00,2019-01-01 04:32:28+00:00,2019-01-01 00:00:00+00:00,-33.954254,151.178041,0.0,48.996091,2.625805,22.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783379,N2893E,NaN,a2f230,N2893E,NaN,K4SD,KRNO,2021-01-31 23:35:43+00:00,2021-01-31 23:47:52+00:00,2021-01-31 00:00:00+00:00,39.626129,-119.909204,1828.8,39.511957,-119.766724,1310.64
1783380,RLJ320,NaN,a36ebd,N320EJ,NaN,KTMB,KBCT,2021-01-31 23:35:45+00:00,2021-01-31 23:49:24+00:00,2021-01-31 00:00:00+00:00,25.651726,-80.361622,914.4,26.403731,-80.068835,266.70
1783381,N4337V,NaN,a531a4,N4337V,NaN,KMIC,KSGS,2021-01-31 23:36:20+00:00,2021-01-31 23:49:14+00:00,2021-01-31 00:00:00+00:00,45.066833,-93.359201,0.0,44.858505,-92.975268,563.88
1783382,N545DS,NaN,a6eadb,N545DS,NaN,KVUO,WA29,2021-01-31 23:36:30+00:00,2021-01-31 23:46:53+00:00,2021-01-31 00:00:00+00:00,45.619720,-122.651099,0.0,45.922852,-122.734636,640.08


In [8]:
yedek_data = flightlist
flightlist = yedek_data

In [16]:
flightlist = flightlist.head(500000000)

In [17]:
from traffic.data import airports
import altair as alt

airports_subset = [
    # Europe
    ["LFPG", "EGLL", "EHAM", "EDDF", "LEMD", "LIRF", "LSZH", "UUEE"],
    # Eastern Asia
    ["VHHH", "RJBB", "RJTT", "RKSI", "RCTP", "RPLL"],
    # Asia (other)
    ["YSSY", "YMML", "OMDB", "VABB", "VIDP", "WSSS"],
    # Americas
    ["CYYZ", "KSFO", "KLAX", "KATL", "KJFK", "SBGR"],
]

data = pd.concat(
    (
        flightlist.query(f'origin == "{airport}"')
        # count the number of departing aircraft per day
        .groupby("day")
        .agg(dict(callsign="count"))
        # label the current chunk with the name of the airport
        .rename(columns=dict(callsign=airport))
        # iterate on all airports in the list hereabove
        for airport in sum(airports_subset, [])
    ),
    axis=1,
)

chart = alt.Chart(
    data.reset_index()
    # prepare data for altair
    .melt("day", var_name="airport", value_name="count")
    # include the name of the city associated with the airport code
    .merge(
        airports.data[["icao", "municipality"]],
        left_on="airport",
        right_on="icao",
        how="left",
    )[["day", "airport", "count", "municipality"]]
    # rename this feature 'city'
    .rename(columns=dict(municipality="city"))
)


def full_chart(source, subset, subset_name):

    # We have many airports, only pick a subset
    chart = source.transform_filter(
        alt.FieldOneOfPredicate(field="airport", oneOf=subset)
    )

    # When we come close to a line, highlight it
    highlight = alt.selection(
        type="single", nearest=True, on="mouseover", fields=["airport"]
    )

    # The scatter plot
    points = (
        chart.mark_point()
        .encode(
            x="day",
            y=alt.Y("count", title="# of departing flights"),
            color=alt.Color("airport", legend=alt.Legend(title=subset_name)),
            # add some legend next to  point
            tooltip=["day", "airport", "city", "count"],
            # not too noisy please
            opacity=alt.value(0.5),
        )
        .add_selection(highlight)
    )

    # The trend plot
    lines = (
        chart.mark_line()
        .encode(
            x="day",
            y="count",
            color="airport",
            size=alt.condition(~highlight, alt.value(1), alt.value(3)),
        )
        # the cloud is a bit messy, draw a trend through it
        .transform_loess("day", "count", groupby=["airport"], bandwidth=0.2)
    )

    return lines + points


# Concatenate several plots
result = alt.vconcat(
    *[
        full_chart(chart, airport_, subset_name).properties(width=600, height=150)
        for subset_name, airport_ in zip(
            [
                "European airports",
                "East-Asian airports",
                "Asian/Australian airports",
                "American airports",
            ],
            airports_subset,
        )
    ]
).resolve_scale(color="independent")

result

alt.VConcatChart(...)